In [ ]:
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
import scanpy as sc
import pandas as pd
import torch
import scipy
import time
from model import *
from utils import *

In [ ]:
scdata = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/simu_sc_counts.csv', index_col = 0)
scdata.head(5)

In [ ]:
scdata.shape

In [ ]:
stdata = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/simu_st_counts.csv', index_col = 0)
stdata.head(5)

In [ ]:
stdata.shape

In [ ]:
scdata = scdata.T
stdata = stdata.T

In [ ]:
stgtcelltype = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/simu_st_celltype.csv', index_col = 0)
stgtcelltype.head(5)

In [ ]:
stgtcelltype.shape

In [ ]:
scdata.head(5)

In [ ]:
stdata.head(5)

In [ ]:
stdata.shape

In [ ]:
spcoor = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/simu_st_metadata.csv', index_col=0)
spcoor.head(5)

In [ ]:
spcoor.shape

In [ ]:
scmetadata = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/metadata.csv', index_col = 0)
scmetadata.head(5)

In [ ]:
scmetadata.shape

In [ ]:
adata = sc.AnnData(scdata, obs=scmetadata)
adata

In [ ]:
adata.X

In [ ]:
adata.var_names

In [ ]:
adata.obs_names

In [ ]:
scdata.head(5)

In [ ]:
scmetadata

In [ ]:
new_scdata = pd.DataFrame(adata.X,index=adata.obs_names,columns=adata.var_names)
new_scdata.head(5)

In [ ]:
adata.obs['embryo']

In [ ]:
scdata.head(5)

In [ ]:
new_scdata.head(5)

In [ ]:
adata.obs

In [ ]:
scdata = new_scdata
stadata = sc.AnnData(stdata)
sc.pp.normalize_total(stadata)
sc.pp.log1p(stadata)
stdata = pd.DataFrame(stadata.X, index = stadata.obs_names, columns = stadata.var_names)

In [ ]:
adata.obsm['spatial'] = scmetadata[['x_global', 'y_global']].values

In [ ]:
adata.obs

In [ ]:
spcoor

In [ ]:
adata.obsm['spatial']

In [ ]:
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
adata.obs

In [ ]:
sc.pp.calculate_qc_metrics(stadata,percent_top=None, log1p=False, inplace=True)


In [ ]:
dp = 1- adata.obs['n_genes_by_counts'].median()/stadata.obs['n_genes_by_counts'].median()
dp

In [ ]:
class setting( object ):
    pass
seed_all(2022)
opt= setting()
setattr(opt, 'device', 'cuda:0') # device
setattr(opt, 'outf', 'log/test') # folder to save log files
setattr(opt, 'n_genes', 351) # number of genes for the input
setattr(opt, 'no_bn', False) # duplicated
setattr(opt, 'lr', 0.002) # learning rate
setattr(opt, 'sigma', 3)  # the spatial variance parameter in the Gaussian function
setattr(opt, 'alpha', 0.8) # MMD loss weight default:0.8
setattr(opt, 'verbose', True) # verbose
setattr(opt, 'mmdbatch', 1000) # batch for MMD loss
setattr(opt, 'dp', dp) # dropout rate for ST data

In [ ]:
opt.device

In [ ]:
if not os.path.exists(opt.outf):
    os.makedirs(opt.outf)

In [ ]:
spcoor

In [ ]:
stgtcelltype

In [ ]:
stadata.obs

In [ ]:
import matplotlib.pyplot as plt
import scanpy as sc

# Set figure size
plt.figure(figsize=(10, 5))

# Create the violin plots
sc.pl.violin(stadata, ['n_genes_by_counts', 'total_counts'], 
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'],jitter=0.4, multi_panel=True)

In [ ]:
st_neighbor = scipy.spatial.distance.cdist(spcoor, spcoor)
sigma = 3
st_neighbor = np.exp(-st_neighbor ** 2/ (2*sigma**2))/ (np.sqrt(2*np.pi)*sigma)

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(spcoor.values[:, 0], spcoor.values[:, 1], c=np.log1p(st_neighbor[400, :]), s=20)

In [ ]:
class setting( object ):
    pass
seed_all(2022)
opt= setting()
setattr(opt, 'device', 'cuda:0')
setattr(opt, 'outf', 'log/Mouse_E1z2')
setattr(opt, 'n_genes', 351)
setattr(opt, 'no_bn', False)
setattr(opt, 'lr', 0.002)
setattr(opt, 'sigma', 3)
setattr(opt, 'alpha', 0.8)
setattr(opt, 'verbose', True)
setattr(opt, 'mmdbatch', 1000)
setattr(opt, 'dp', dp)

In [ ]:
%%time
testmodel = SOmodel(opt)
testmodel.togpu()
loss_curve = testmodel.train_wholedata(400, torch.tensor(scdata.values).float(), torch.tensor(stdata.values).float(), torch.tensor(spcoor.values).float())

In [ ]:
plt.figure(figsize=(10,6))
skip = 10
N = len(loss_curve['E']) - skip
plt.plot(np.arange(N), loss_curve['E'][skip:], label='E')
plt.plot(np.arange(N), loss_curve['E_pred'][skip:], label='E_pred')
plt.plot(np.arange(N), loss_curve['E_circle'][skip:], label='E_circle')
plt.plot(np.arange(N), loss_curve['E_mmd'][skip:], label='E_mmd')
plt.legend()

In [ ]:
testmodel.modeleval()
scembedding = testmodel.netE(torch.tensor(scdata.values, dtype=torch.float32).cuda())
stembedding = testmodel.netE(torch.tensor(stdata.values, dtype=torch.float32).cuda())
netst2sc = F.softmax(stembedding.mm(scembedding.t()),dim=1).detach().cpu().numpy()
netsc2st = F.softmax(scembedding.mm(stembedding.t()), dim=1).detach().cpu().numpy()

In [ ]:
netst2sc.shape

In [ ]:
netsc2st.shape

In [ ]:
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
dataset_name = 'E1z2'
result_dir = f'result/{timestamp}_{dataset_name}'
os.makedirs(result_dir, exist_ok=True)

np.save(f'{result_dir}/st2sc_mapping.npy', netst2sc)
np.save(f'{result_dir}/sc2st_mapping.npy', netsc2st)
np.save(f'{result_dir}/sc_embedding.npy', scembedding.detach().cpu().numpy())
np.save(f'{result_dir}/st_embedding.npy', stembedding.detach().cpu().numpy())

In [ ]:
testmodel.save()

In [ ]:
def all_coord(mappingmtx,spcoor):
    mappingnorm = mappingmtx.div(mappingmtx.sum(axis=1), axis=0)

    spaotsc_coord = np.zeros([mappingnorm.shape[0],2])
    spaotsc_coord[:,0] = np.matmul(mappingnorm.values,spcoor.coord_x.values)
    spaotsc_coord[:,1] = np.matmul(mappingnorm.values,spcoor.coord_y.values)
    return spaotsc_coord

def k_coord(mappingmtx,spcoor, k=1):
    maskmap = np.zeros_like(mappingmtx)

    argmask = np.argsort(1/mappingmtx.values,axis=1)

    for i in range(argmask.shape[0]):
        for j in range(k):
            maskmap[i,argmask[i,j]]=1

    mappingnorm = (maskmap*mappingmtx).div((maskmap*mappingmtx).sum(axis=1), axis=0)

    spaotsc_coord = np.zeros([mappingnorm.shape[0],2])
    spaotsc_coord[:,0] = np.matmul(mappingnorm.values,spcoor.coord_x.values)
    spaotsc_coord[:,1] = np.matmul(mappingnorm.values,spcoor.coord_y.values)
    return spaotsc_coord

In [ ]:
adata.obsm['SpatialDA'] = all_coord(pd.DataFrame(netsc2st, index=adata.obs_names, columns=stadata.obs_names), spcoor)

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

In [ ]:
def gen_sc_adj_matrix(sc_embeddings):
    #change to numpy array if tensor
    if isinstance(sc_embeddings, torch.Tensor):
        embeddings = sc_embeddings.detach().cpu().numpy()
    else:
        embeddings = sc_embeddings

    #calculate inner product
    adj_matrix = np.matmul(embeddings, embeddings.T)

    #normalize it
    # row_sums = adj_matrix.sum(axis=1)
    # normalized_adj_matrix = adj_matrix / row_sums[:, np.newaxis]
    normalized_adj_matrix = adj_matrix / adj_matrix.sum(axis=1, keepdims=True)

    return normalized_adj_matrix

# def recover_coord_from_adj_matrix(adj_matrix, n_components=2):
#     from sklearn.manifold import MDS

#     distance_matrix = 1 - adj_matrix
#     distance_matrix = 0.5 * (distance_matrix + distance_matrix.T)

#     #apply mds to recover coordinates
#     mds = MDS(n_components=n_components,
#               dissimilarity='precomputed',
#               random_state=42,
#               normalized_stress='auto')
    
#     coordinates = mds.fit_transform(distance_matrix)

#     return coordinates

def recover_coord_from_adj_matrix(adj_matrix, sigma=3.0, n_components=2, method='smacof', max_iter=300):
    from sklearn.manifold import MDS
    import warnings

In [191]:
#generate sc embeddings
scembedding = testmodel.netE(torch.tensor(scdata.values, dtype = torch.float32).cuda())

#generate sc adjacency matrix from embeddings
sc_adjacency_matrix = gen_sc_adj_matrix(scembedding)

#recover coordinates from adjacency matrix
recovered_coordinates = recover_coord_from_adj_matrix(sc_adjacency_matrix)

#add to AnnData object
adata.obsm['MDSSpatial'] = recovered_coordinates

In [193]:
adata.obsm['MDSSpatial']

array([[-0.16331454,  0.75031373],
       [ 0.43977416,  0.16300028],
       [-0.50681446, -0.49988661],
       ...,
       [-0.23938644, -0.60675617],
       [ 0.40582559,  0.06321056],
       [ 0.44911275, -0.60269493]])

In [194]:
adata.obsm['SpatialDA']

array([[1627.29216741, 1349.74699412],
       [1625.44197922, 1348.43905111],
       [1625.99117369, 1336.13772766],
       ...,
       [1550.36125401, 1536.50573694],
       [1556.35831   , 1521.93157017],
       [1553.71221347, 1518.64542879]])

In [225]:
def distance_from_adjacency(adj_matrix, sigma = 3.0, epsilon=1e-10):
    #assuming adjacency matrix is normalized so that each row sums to 1

    #adjacecny values should be pisitive to avoid log(0)
    adj_cleaned = np.maximum(adj_matrix, epsilon)

    #apply inverse gaussian kernel transformation
    #D_{ij} = sqrt(-2*sigma^2 * ln(S^{SC}_{ij} + epsilon))
    distance_squared = -2 * sigma**2 * np.log(adj_cleaned)

    #ensure distances are non-negative
    distance_squared = np.maximum(distance_squared, 0)

    #take square root to get distance
    distance_matrix = np.sqrt(distance_squared)

    #make the matrix symmetric (average with its transpose)
    distance_matrix = 0.5 * (distance_matrix + distance_matrix.T)

    #check range of distances
    dist_min, dist_max = np.min(distance_matrix), np.max(distance_matrix)
    print(f'distance range: [{dist_min:.4f}, {dist_max:.4f}]')

    #if range too wide, scale distances to reasonable range
    #will help mds converge faster
    if dist_max > 100 * dist_min:
        scaling_factor = 10.0/ dist_max
        distance_matrix *= scaling_factor
        print(f'scaled distances by factor {scaling_factor:.6f}')

    return distance_matrix


def coordinates_from_adjacency(adj_matrix, sigma=3.0, n_components=2, max_iter = 300):
    from sklearn.manifold import MDS
    from sklearn.decomposition import PCA
    #recover distance from adjacency
    distance_matrix = distance_from_adjacency(adj_matrix,  sigma)

    #appl mds to recover coordinates
    initial_coords = PCA(n_components=n_components).fit_transform(distance_matrix)
    mds = MDS(n_components=n_components,
              dissimilarity='precomputed',
              random_state = 42,
              normalized_stress= 'auto',
              metric = False,
              max_iter = max_iter,
              n_init = 1,
              eps = 1e-5,
              init = initial_coords)
    
    try:
        print('running MDS.....')
        coordinates = mds.fit_transform(distance_matrix)
        print(f'MDS completed in {mds.n_iter_} iterations')
        print(f'value of stress is {mds.stress_}')
        return coordinates
    except Exception as e:
        error_message = f"MDS failed: {e}"
        print(error_message)
        print("Distance matrix statistics:")
        print(f"  Shape: {distance_matrix.shape}")
        print(f"  Min: {np.min(distance_matrix):.6e}")
        print(f"  Max: {np.max(distance_matrix):.6e}")
        print(f"  Mean: {np.mean(distance_matrix):.6e}")
        print(f"  Median: {np.median(distance_matrix):.6e}")
        print(f"  Contains NaN: {np.isnan(distance_matrix).any()}")
        print(f"  Contains Inf: {np.isinf(distance_matrix).any()}")

        # Return the error instead of falling back to a simpler approach
        raise ValueError(f"MDS failed with Gaussian kernel approach: {error_message}. "
                     f"Consider increasing max_iter, adjusting sigma, or using a different approach.")
    
    # coordinates = mds.fit_transform(distance_matrix)

    # return coordinates

def sc_coordinates_from_embeddings(sc_embeddings, sigma=3.0, max_iter = 300):
    if isinstance(sc_embeddings, torch.Tensor):
        embeddings = sc_embeddings.detach().cpu().numpy()
    else:
        embeddings = sc_embeddings

    #calculate SC spatial adjacency matrix using inner products 
    sc_adjacency_matrix = np.matmul(embeddings, embeddings.T)

    #normalize rows to sum to 1
    row_sums = sc_adjacency_matrix.sum(axis=1, keepdims=True)
    sc_adjacency_matrix_norm = sc_adjacency_matrix / row_sums

    #recover coordinates using inverse gaussian kernel and MDS
    coordinates = coordinates_from_adjacency(sc_adjacency_matrix_norm, sigma = 3.0, max_iter = max_iter)

    return coordinates, sc_adjacency_matrix_norm

In [226]:
#generate SC embeddings
testmodel.modeleval()
scembedding = testmodel.netE(torch.tensor(scdata.values, dtype=torch.float32).cuda())

#get coordinates from embeddings 
recovered_coords, sc_adjacency_matrix = sc_coordinates_from_embeddings(scembedding, sigma = opt.sigma, max_iter = 100)

distance range: [12.4292, 13.4399]


TypeError: MDS.__init__() got an unexpected keyword argument 'init'

In [220]:
recovered_coords

array([[-0.11206713,  0.45554845],
       [ 0.260868  ,  0.10569503],
       [-0.32683856, -0.32333828],
       ...,
       [-0.24532733, -0.48552892],
       [ 0.29564756,  0.05532238],
       [ 0.5017872 , -0.59226047]])

In [217]:
#generate SC embeddings
testmodel.modeleval()
scembedding = testmodel.netE(torch.tensor(scdata.values, dtype=torch.float32).cuda())

#get coordinates from embeddings 
recovered_coords_1, sc_adjacency_matrix = sc_coordinates_from_embeddings(scembedding, sigma = opt.sigma, max_iter = 500)

distance range: [12.4292, 13.4399]
running MDS.....
MDS completed in 2 iterations
value of stress is 0.3023023554146359


In [210]:
#generate SC embeddings
testmodel.modeleval()
scembedding = testmodel.netE(torch.tensor(scdata.values, dtype=torch.float32).cuda())

#get coordinates from embeddings 
recovered_coords_2, sc_adjacency_matrix = sc_coordinates_from_embeddings(scembedding, sigma = opt.sigma, max_iter = 800)

distance range: [12.4292, 13.4399]
running MDS.....
MDS completed in 800 iterations


In [211]:
recovered_coords_2

array([[-1.57309198,  7.51784355],
       [ 7.20429536,  2.87709859],
       [-7.1595635 , -7.08994888],
       ...,
       [-3.66975646, -9.12048066],
       [ 5.28815358,  0.90581709],
       [ 6.04676095, -8.0485039 ]])

In [209]:
recovered_coords_1

array([[-1.91521344,  8.02177978],
       [ 6.48167989,  2.5084535 ],
       [-7.12569612, -7.13461656],
       ...,
       [-3.6721057 , -9.18012475],
       [ 4.97039548,  0.64029698],
       [ 6.08493159, -8.04453808]])

In [202]:
recovered_coords

array([[-1.9156618 ,  8.5463558 ],
       [ 5.82427083,  2.42566554],
       [-7.20588915, -7.10971964],
       ...,
       [-3.78128498, -9.19497135],
       [ 4.89969266,  0.73432087],
       [ 6.10946935, -8.07416432]])

In [199]:
scembedding

tensor([[ 0.1940, -1.2524,  0.0839,  ..., -0.4363,  0.3484, -0.0688],
        [ 0.3140, -0.9275, -0.0646,  ..., -0.4137,  0.4129,  0.1386],
        [ 0.2854, -1.0332, -0.5846,  ...,  0.1499,  0.5140, -0.3145],
        ...,
        [-0.1881, -1.8073, -0.7278,  ..., -1.2491,  0.2379, -0.2599],
        [ 0.3583, -1.7920, -0.2019,  ..., -1.0905, -0.0391,  0.2596],
        [ 0.5933, -1.4357, -0.1443,  ..., -0.7041,  0.0051,  0.1786]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [184]:
adj_matrix = np.array([
    [2, 4, 6], 
    [1, 3, 5], 
    [7, 8, 9]
])


In [185]:
row_sums = adj_matrix.sum(axis=1)
row_sums


array([12,  9, 24])

In [186]:
adj_matrix = adj_matrix / row_sums[:, np.newaxis]

In [188]:
row_sums[np.newaxis, :]

array([[12,  9, 24]])

In [195]:
spcoor

,coord_x,coord_y
ST_12,1581.242732,1329.313864
ST_13,1586.662026,1329.313864
ST_42,1581.242732,1334.863919
ST_43,1586.662026,1334.863919
ST_49,1619.177788,1334.863919
...,...,...
ST_1182,1581.242732,1545.766004
ST_1183,1586.662026,1545.766004
ST_1184,1592.081319,1545.766004
ST_1185,1597.500613,1545.766004


In [ ]:
def diffusion_map_coordinates(similarity_matrix, n_components=2, t=1):
    '''
    convert similaruty matrix to coordinates using diffusion maps
    '''
    import numpy as np
    from scipy import linalg
    import time

    start_time = time.time()

    #check similairyt matrix is already row-normalzied
    row_sums = similarity_matrix.sum(axis=1)
    is_normalized = np.allclose(row_sums, 1.0, rtol = 1e-5)

    if not is_normalized:
        print('normalizing similarity matrix...')
        #normalize rows to sum to 1 (create Markov transition matrix P)
        D_inv = np.diag(1.0 / row_sums)
        P = D_inv @ similarity_matrix
    else:
        print('similarity matrix is already normalized')
        P = similarity_matrix

    #compute eigenvectors and eigenvalues
    print('computing eigenvectors and eignevalues....')
    try:
        # scipy.linalg.eigh for symmetric matrices (faster and more stable)
        # if matrix not symmetric, make it symmetric first
        P_sym = 0.5 * (P + P.T)
        eigenvalues, eigenvectors = linalg.eigh(P_sym)
    except:
        #fallback to general case
        eigenvalues, eigenvectors = linalg.eig(P)
        #ensure real parts only cause eigenvals should be real for markov matrices
        eigenvalues = np.real(eigenvalues)
        eigenvectors = np.real(eigenvectors)


    #sort eigenvalues and eignevectors in descending order of eigenvalues
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    print(f'largest eigenvalues: {eigenvalues[:n_components+1]}')

    #create the diffusion map embedding
    #skip the first eigenvector (corresponds to eigenvalue 1) as it is constant
    embedding = np.zeros((similarity_matrix.shape[0], n_components))

    for i in range(n_components):
        #use eigenvalues and eigenvectors from index 1 onwards
        embedding[:, i] = eigenvalues[i+1] ** t * eigenvectors[:, i+1]

    print(f'diffusion map computed in {time.time() - start_time: .2f} seconds')

    return embedding

def sc_coordinates_from_diffusion_embeddings(sc_embeddings, n_components=2):
    '''use diffusion maps to generate spatial embeddings from SC embeddings'''
    start_time = time.time()

    if isinstance(sc_embeddings, torch.Tensor):
        embeddings = sc_embeddings.detach().cpu().numpy()
    else:
        embeddings = sc_embeddings

    print(f'embeddings shape: {embeddings.shape}')

